In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

from torch import optim
from torch.utils.data import DataLoader
from torch import nn

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# system paramater
Mc = 8
Mr = 4
N = 36
K = 1

Pc_dB = 10
Pr_dB = 10
tau_dB = 2
Pc = 10**(Pc_dB/10)
Pr = 10**(Pr_dB/10)
tau = 10**(tau_dB/10)

BS_loc = np.array([-100,100])
RIS_loc = np.array([0,0])
Radar_loc = np.array([100,100])
UE_center = [0,10]
UE_range = 5

kappa = 100
v = 0.0001

# training parameter
sample_num = 5000
batch_size = 50
learning_rate = 1e-4

In [4]:
def ULA(M,angle): # angle is not rad
    return np.sqrt(1/M)*np.matrix(np.exp(1j*np.pi*np.sin(angle*np.pi/180)*np.linspace(0,M-1,M))).T

def LOS_PL(d,fc): 
    return 22*np.log10(d)+28+20*np.log10(fc)

def NLOS_PL(d,fc): 
    return 36.7*np.log10(d)+22.7+26*np.log10(fc)

In [5]:
UE_radius = UE_range*np.random.rand(sample_num,1)
UE_direction = 2*np.pi*np.random.rand(sample_num,1)
UE_x = UE_center[0]+UE_radius*np.cos(UE_direction)
UE_y = UE_center[1]+UE_radius*np.sin(UE_direction)
UE_loc = np.concatenate((UE_x,UE_y),axis=1)

In [6]:
# distance
BS_Rad_dist = np.linalg.norm(BS_loc-Radar_loc)
BS_RIS_dist = np.linalg.norm(BS_loc-RIS_loc)
RIS_Rad_dist = np.linalg.norm(RIS_loc-Radar_loc)
RIS_UE_dist = np.linalg.norm(UE_loc-RIS_loc,axis=1)
Rad_UE_dist = np.linalg.norm(UE_loc-Radar_loc,axis=1)
BS_UE_dist = np.linalg.norm(UE_loc-BS_loc,axis=1)

In [7]:
# angle (终点-起点求arc) 
angle1 = -np.arctan((RIS_loc[1]-BS_loc[1])/(RIS_loc[0]-BS_loc[0]))*180/np.pi-45 # BS->RIS o
angle2 = -np.arctan((BS_loc[1]-RIS_loc[1])/(BS_loc[0]-RIS_loc[0]))*180/np.pi-90 # RIS->BS o

angle3 = -np.arctan((Radar_loc[1]-BS_loc[1])/(Radar_loc[0]-BS_loc[0]))*180/np.pi-45 # BS->Rad o
angle4 = -np.arctan((BS_loc[1]-Radar_loc[1])/(BS_loc[0]-Radar_loc[0]))*180/np.pi # Rad->BS o

angle5 = -np.arctan((RIS_loc[1]-Radar_loc[1])/(RIS_loc[0]-Radar_loc[0]))*180/np.pi # Rad->RIS o
angle6 = -np.arctan((Radar_loc[1]-RIS_loc[1])/(Radar_loc[0]-RIS_loc[0]))*180/np.pi+90 # RIS->Rad o

angle7 = -np.arctan((UE_loc[:,1]-BS_loc[1])/(UE_loc[:,0]-BS_loc[0]))*180/np.pi-45 # BS->UE o
angle8 = -np.arctan((UE_loc[:,1]-Radar_loc[1])/(UE_loc[:,0]-Radar_loc[0]))*180/np.pi # Rad->UE  o
angle9 = np.zeros(sample_num) # RIS->UE o
for i in range(sample_num):
    if (UE_loc[i,0]-RIS_loc[0])!=0:
        angle9[i] = np.arctan((UE_loc[i,1]-RIS_loc[1])/(UE_loc[i,0]-RIS_loc[0]))*180/np.pi # 避免90度的情况
    else:
        angle9[i] = 90.
angle9[angle9>0] = -angle9[angle9>0]+90
angle9[angle9<0] = -angle9[angle9<0]-90

In [8]:
# pathloss
fc = 2.4 # GHz, carrier freq
n_pw_sp = 58 # Noise power spectral density
dc_pl_0 = np.sqrt(10**((n_pw_sp-NLOS_PL(BS_UE_dist,fc))/10))
dc_pl = np.expand_dims(dc_pl_0,axis=[1,2])
dr_pl_0 = np.sqrt(10**((n_pw_sp-NLOS_PL(Rad_UE_dist,fc))/10))
dr_pl = np.expand_dims(dr_pl_0,axis=[1,2])
Hc_pl = np.sqrt(10**((n_pw_sp-LOS_PL(BS_RIS_dist,fc)) /10))
Hr_pl = np.sqrt(10**((n_pw_sp- LOS_PL(RIS_Rad_dist,fc))  /10))
h_pl_0 = np.sqrt(10**((n_pw_sp- LOS_PL(RIS_UE_dist,fc)) /10))
h_pl = torch.from_numpy(np.expand_dims(h_pl_0,axis=[1,2]))
Hint_pl = np.sqrt(10**((n_pw_sp-LOS_PL(BS_Rad_dist,fc)) /10))

In [9]:
dc = dc_pl*np.sqrt(1/2)*(np.random.randn(sample_num,K,Mc)+1j*np.random.randn(sample_num,K,Mc))
dr = dr_pl*np.sqrt(1/2)*(np.random.randn(sample_num,K,Mr)+1j*np.random.randn(sample_num,K,Mr))

Hc_LOS_0 = ULA(N,angle2)*ULA(Mc,angle1).H
Hc_LOS = np.tile(np.expand_dims(Hc_LOS_0,axis=0),(sample_num,1,1))
Hc_NLOS = (np.random.randn(sample_num,N,Mc)+1j*np.random.randn(sample_num,N,Mc))
Hc = Hc_pl*(np.sqrt(kappa/(kappa+1))*Hc_LOS+np.sqrt(1/(kappa+1))*Hc_NLOS) # BS->RIS

Hr_LOS_0 = ULA(N,angle6)*ULA(Mr,angle5).H
Hr_LOS = np.tile(np.expand_dims(Hr_LOS_0,axis=0),(sample_num,1,1))
Hr_NLOS = (np.random.randn(sample_num,N,Mr)+1j*np.random.randn(sample_num,N,Mr))
Hr = Hr_pl*(np.sqrt(kappa/(kappa+1))*Hr_LOS+np.sqrt(1/(kappa+1))*Hr_NLOS) # Radar->RIS

h_LOS_0 = np.zeros((sample_num,N))+1j*np.zeros((sample_num,N))
for i in range(sample_num):
    h_LOS_0[i,:] = ULA(N,angle9[i]).H
h_LOS = np.expand_dims(h_LOS_0,axis=1)
h_NLOS = (np.random.randn(sample_num,K,N)+1j*np.random.randn(sample_num,K,N))
h = h_pl*(np.sqrt(kappa/(kappa+1))*h_LOS+np.sqrt(1/(kappa+1))*h_NLOS) # RIS-> UE

Hint_LOS_0 = ULA(Mr,angle4)*ULA(Mc,angle3).H
Hint_LOS = np.tile(np.expand_dims(Hint_LOS_0,axis=0),(sample_num,1,1))
Hint_NLOS = (np.random.randn(sample_num,Mr,Mc)+1j*np.random.randn(sample_num,Mr,Mc))
Hint = Hint_pl*(np.sqrt(kappa/(kappa+1))*Hint_LOS+np.sqrt(1/(kappa+1))*Hint_NLOS) # BS->Rad

target_angle = 30
H_tg = 0.01*ULA(Mr,target_angle)*ULA(Mr,target_angle).H

In [183]:
class mynet(nn.Module):
    def __init__(self):
        super().__init__()
        self.R1CNN = nn.Sequential(
            nn.Conv2d(2,36,kernel_size=3, stride=1, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=3,stride=2)
            
            ,nn.Conv2d(36,64,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=3,stride=2)
            
            ,nn.Conv2d(64,128,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            
            ,nn.Conv2d(128,36,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
        ).to(device)
        
        self.R2CNN = nn.Sequential(
            nn.Conv2d(2,36,kernel_size=3, stride=1, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=3,stride=2)
            
            ,nn.Conv2d(36,64,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=3,stride=2)
            
            ,nn.Conv2d(64,128,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            
            ,nn.Conv2d(128,36,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
        ).to(device)
        
        self.R3CNN = nn.Sequential(
            nn.Conv2d(74,256,kernel_size=3, stride=1, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=3,stride=2)
            
            ,nn.Conv2d(256,512,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=2,stride=1)
            
            ,nn.Conv2d(512,1024,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
            ,nn.MaxPool2d(kernel_size=2,stride=1)
            
            ,nn.Conv2d(1024,2048,kernel_size=3, stride=2, padding=2)
            ,nn.ReLU(inplace=True)
        ).to(device)
        
        self.phase_layer = nn.Sequential(
            nn.Linear(8840, 2048),# 2(KN+KMr+KMc+MrMc+McN+MrN)
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(2048),

            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),

            nn.Linear(256, 196),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(196),

            nn.Linear(196, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(128),

            nn.Linear(128, 36),
            
            nn.Softmax(dim=1)
        ).to(device)

        self.w_dnn_layer = nn.Sequential(
            nn.Linear(80, 1024),  # input:K*Mc*2(to UE)+Mc*Mr*2(to radar)
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(1024),

            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(64),

            nn.Linear(64, 16)  # output:2*Mc
        ).to(device)

    def forward(self, R1, R2, R3, dc_torch, dr_torch, Hc_torch, Hr_torch, h_torch, Hint_torch, device):
        x1 = self.R1CNN(R1)
        x1 = torch.flatten(x1,start_dim=1)
        
        x2 = self.R2CNN(R2)
        x2 = torch.flatten(x2,start_dim=1)

        x3 = self.R3CNN(R3)
        x3 = torch.flatten(x3,start_dim=1)
        
        x = torch.cat((x1,x2,x3),1)
        
        angle = self.phase_layer(x)
        theta = torch.exp(1j*2*np.pi*angle)

        batch = theta.shape[0]
        Mc = dc_torch.shape[2]
        Mr = dr_torch.shape[2]
        K = 1
        N = Hc_torch.shape[1]

        Hcom = torch.zeros(dc_torch.shape) + 1j * torch.zeros(dc_torch.shape)
        for i in range(batch):
            Hcom[i, :, :] = dc_torch[i, :, :] + h_torch[i, :, :] @ torch.diag(theta[i, :]) @ Hc_torch[i, :, :]

        Hint_ = torch.zeros(Hint_torch.shape) + 1j * torch.zeros(Hint_torch.shape)
        for i in range(batch):
            Hint_[i, :, :] = Hint_torch[i, :, :].to(device) + Hr_torch[i, :, :].conj().T.to(device) @ torch.diag(theta[i, :]).to(device) @ Hc_torch[i, :, :].to(device)

        a = Hcom.view(batch, -1)
        b = Hint_.view(batch, -1)

        H_sep = torch.cat((torch.real(a), torch.imag(a), torch.real(b), torch.imag(b)), 1)
        
        w = self.w_dnn_layer(H_sep.to(device))
        
        w_real_ = w[:, :Mc * K]
        w_imag_ = w[:, Mc * K:]
        w_complex = w_real_ + 1j * w_imag_
        
        sqrt_power = torch.linalg.vector_norm(w_complex,dim=1)
        w_final_temp = torch.reshape(w_complex, [batch, Mc, K])
        w_final = torch.zeros((w_final_temp.shape))+1j*torch.zeros((w_final_temp.shape))
        for i in range(len(sqrt_power)):
            w_final[i] = w_final_temp[i]/sqrt_power[i] * np.sqrt(Pc)
        
        return theta, w_final


In [184]:
def extract_feature(dc,dr,Hc,Hr,h,Hint):
    sample_num = dc.shape[0]
    N = h.shape[2]
    K = h.shape[1]
    Mr = dr.shape[2]
    Mc = dc.shape[2]
    
    R1 = torch.zeros((sample_num,1,N+1,N+1))+1j*torch.zeros((sample_num,1,N+1,N+1))
    R2 = torch.zeros((sample_num,1,N+1,N+1))+1j*torch.zeros((sample_num,1,N+1,N+1))
    R3 = torch.zeros((sample_num,N+1,Mr,Mc))+1j*torch.zeros((sample_num,N+1, Mr,Mc))
    for i in range(sample_num):
        phi1 = torch.diag(h[i,:,:].squeeze()) @ Hc[i,:,:]
        phidc = phi1 @ dc[i,:,:].conj().T
        dcphi = dc[i,:,:] @ phi1.conj().T
        a1 = np.c_[phi1 @ phi1.conj().T, phidc]
        b1 = np.c_[dcphi, 0] 
        R1[i,:,:,:] = torch.from_numpy(np.r_[a1,b1])

        phi2 = torch.diag(h[i,:,:].squeeze()) @ Hr[i,:,:]
        phidr = phi2 @ dr[i,:,:].conj().T
        drphi = dr[i,:,:] @ phi2.conj().T
        a2 = np.c_[phi2 @ phi2.conj().T, phidr]
        b2 = np.c_[drphi, 0]
        R2[i,:,:,:] = torch.from_numpy(np.r_[a2,b2])

        for l in range(N):
            Hr_H = Hr[i,:,:].conj().T       
            R3[i,l,:,:] = Hr_H[:,l].reshape(-1,1) @ Hc[i,l,:].reshape(1,-1)

        R3[i,-1,:,:] = Hint[i,:,:]
       
    R1_real = torch.real(R1)
    R1_imag = torch.imag(R1)
    R2_real = torch.real(R2)
    R2_imag = torch.imag(R2)
    R3_real = torch.real(R3)
    R3_imag = torch.imag(R3)
    
    return R1_real,R1_imag,R2_real,R2_imag,R3_real,R3_imag

In [185]:
def recover_chan(x,K,Mr,Mc,N):
    batch = x.shape[0]
    
    dr = (x[:, 0:K * Mr] + 1j * x[:, K * Mr:2 * (K * Mr)]).reshape(batch,K,Mr)
    dc = (x[:, 2 * (K * Mr):2 * (K * Mr) + (K * Mc)] + 1j * x[:, (2 * (K * Mr) + (K * Mc)):2 * (K * Mc + K * Mr)]).reshape(batch,K,Mc)

    Hr = (x[:, 2 * (K * Mc + K * Mr):2 * (K * Mc + K * Mr) + (N * Mr)] \
           + 1j * x[:, 2 * (K * Mc + K * Mr) + (N * Mr):2 * (K * Mc + K * Mr + N * Mr)]).reshape(batch,N,Mr)
    Hc = (x[:, 2 * (K * Mc + K * Mr + N * Mr):2 * (K * Mc + K * Mr + N * Mr) + (N * Mc)] \
           + 1j * x[:, 2 * (K * Mc + K * Mr + N * Mr) + (N * Mc):2 * (K * Mc + K * Mr + N * Mr + N * Mc)]).reshape(batch,N,Mc)

    h = (x[:, 2 * (K * Mc + K * Mr + N * Mr + N * Mc):2 * (K * Mc + K * Mr + N * Mr + N * Mc) + (K * N)] \
          + 1j * x[:,2 * (K * Mc + K * Mr + N * Mr + N * Mc) + (K * N):2 * (K * Mc + K * Mr + N * Mr + N * Mc + K * N)]).reshape(batch,K,N)
    Hint = (x[:,2 * (K * Mc + K * Mr + N * Mr + N * Mc + K * N):2 * (K * Mc + K * Mr + N * Mr + N * Mc + K * N) \
                    + (Mc * Mr)] + 1j * x[:, 2 * (K * Mc + K * Mr + N * Mr + N * Mc + K * N) + (Mc * Mr):]).reshape(batch,Mr,Mc)
    
    return dc,dr,Hc, Hr,h,Hint

In [186]:
def get_snr(theta, w, dc, dr, Hc, Hr, h, Hint, Htg, Pc, Pr):
    batch_size = dc.shape[0]
    Mc = dc.shape[2]
    N = h.shape[2]
    
    awgn_var = 1e-4
        
    snrc = torch.zeros(batch_size,1)
    snrr = torch.zeros(batch_size,1)
    
    for b in range(batch_size):
        Theta = torch.diag(theta[b,:])
        
        Hcom = dc[b,:,:] + h[b,:,:] @ Theta @ Hc[b,:,:]
        Hradint = dr[b,:,:] + h[b,:,:] @ Theta @ Hr [b,:,:]   
        snrc_nom = (torch.linalg.vector_norm(Hcom @ w[b,:,:])) ** 2  
        snrc_dem = Pr * (torch.linalg.matrix_norm(Hradint))** 2 + awgn_var
        
        snrc[b] = snrc_nom / snrc_dem
        
        Hint_all = Hint[b,:,:] + Hr[b,:,:].conj().T @ Theta @ Hc[b,:,:]
        snrr_dem = awgn_var + torch.linalg.vector_norm(Hint_all @ w[b,:,:]) ** 2
        snrr_nom = Pr*torch.linalg.matrix_norm(Htg)**2
        snrr[b] = snrr_nom / snrr_dem

    return torch.mean(snrc),torch.mean(snrr)

In [187]:
# make samples
dr_vec = np.reshape(dr,(sample_num,-1))
dc_vec = np.reshape(dc,(sample_num,-1))
Hr_vec = np.reshape(Hr,(sample_num,-1))
Hc_vec = np.reshape(Hc,(sample_num,-1))
h_vec = np.reshape(h,(sample_num,-1))
Hint_vec = np.reshape(Hint,(sample_num,-1))

cascade = (np.real(dr_vec),np.imag(dr_vec),
           np.real(dc_vec),np.imag(dc_vec),
           np.real(Hr_vec),np.imag(Hr_vec),
           np.real(Hc_vec),np.imag(Hc_vec),
           np.real(h_vec),np.imag(h_vec),
           np.real(Hint_vec),np.imag(Hint_vec))
samples = np.concatenate(cascade,axis=-1).astype(np.float32)

trainloader = DataLoader(samples, shuffle=True, batch_size=batch_size)

In [188]:
mymodel = mynet()
optimizer = optim.Adam(mymodel.parameters(), lr=learning_rate)

mymodel.to(device)

mynet(
  (R1CNN): Sequential(
    (0): Conv2d(2, 36, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(36, 64, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (7): ReLU(inplace=True)
    (8): Conv2d(128, 36, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (9): ReLU(inplace=True)
  )
  (R2CNN): Sequential(
    (0): Conv2d(2, 36, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(36, 64, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_m

In [189]:
epochs = 3
losses = []
Htg_torch = torch.from_numpy(H_tg).to(torch.complex64)
mymodel.train()
for e in range(epochs):
    running_loss = 0
    for chan_ in trainloader:
        chan = chan_.to(device)
        dc_32,dr_32,Hc_32, Hr_32,h_32,Hint_32 = recover_chan(chan,K,Mr,Mc,N)
        R1_real,R1_imag,R2_real,R2_imag,R3_real,R3_imag = extract_feature(dc_32,dr_32,Hc_32, Hr_32,h_32,Hint_32)
        
        R1 = torch.cat((R1_real,R1_imag),1)
        R2 = torch.cat((R2_real,R2_imag),1)
        R3 = torch.cat((R3_real,R3_imag),1)
        # 变torch
#         R1 = R1_.to(torch.complex64)
#         R2 = R2_.to(torch.complex64)
#         R3 = R3_.to(torch.complex64)
        
        dr_ = dr_32.to(torch.complex64)
        dc_ = dc_32.to(torch.complex64)
        Hr_ = Hr_32.to(torch.complex64)
        Hc_ = Hc_32.to(torch.complex64)
        h_ = h_32.to(torch.complex64)
        Hint_ = Hint_32.to(torch.complex64)
        
        theta,w = mymodel(R1, R2, R3, dc_, dr_, Hc_, Hr_, h_, Hint_, device)
        
        snrc,snrr = get_snr(theta,w, dc_, dr_, Hc_, Hr_, h_, Hint_, Htg_torch, Pc, Pr)
        loss = -snrc + np.max([0,-snrr+tau])
        print(loss)
        loss.requires_grad_(True)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        losses.append(running_loss / len(trainloader))
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            e + 1,
            running_loss / len(trainloader)))

tensor(0.6214)
tensor(0.8628)
tensor(0.6770)
tensor(0.8297)
tensor(0.8037)
tensor(0.6519)
tensor(0.9110)
tensor(0.7287)
tensor(0.6908)
tensor(0.8740)
tensor(0.6474)
tensor(0.7742)
tensor(0.8110)
tensor(0.8055)
tensor(0.7579)
tensor(0.7289)
tensor(0.6957)
tensor(0.8669)
tensor(0.8361)
tensor(0.6448)
tensor(0.7310)
tensor(0.8538)
tensor(0.8631)
tensor(0.9071)
tensor(0.9653)
tensor(0.8160)
tensor(0.6691)
tensor(0.6564)
tensor(0.8012)
tensor(0.7908)
tensor(0.8528)
tensor(0.8540)
tensor(0.8045)
tensor(0.8612)
tensor(0.7570)
tensor(0.6846)
tensor(0.8245)
tensor(0.7705)
tensor(0.9191)
tensor(0.7355)
tensor(0.8279)
tensor(0.8004)
tensor(0.7272)
tensor(0.6804)
tensor(0.9021)
tensor(0.6891)
tensor(0.8129)
tensor(0.7337)
tensor(0.8562)
tensor(0.8141)
tensor(0.9286)
tensor(0.8802)
tensor(0.9003)
tensor(0.6453)
tensor(0.8068)
tensor(0.8654)
tensor(0.8346)
tensor(0.8185)
tensor(0.7814)
tensor(0.7109)
tensor(0.7113)
tensor(0.7945)
tensor(0.8527)
tensor(0.7784)
tensor(0.6990)
tensor(0.8123)
tensor(0.6

KeyboardInterrupt: 

In [74]:
chan = next(iter(trainloader))
dc_32,dr_32,Hc_32, Hr_32,h_32,Hint_32 = recover_chan(chan,K,Mr,Mc,N)
R1_real,R1_imag,R2_real,R2_imag,R3_real,R3_imag = extract_feature(dc_32,dr_32,Hc_32, Hr_32,h_32,Hint_32)

In [76]:
R1_real.shape

torch.Size([50, 1, 37, 37])

In [77]:
R1_imag.shape

torch.Size([50, 1, 37, 37])

In [136]:
R3.shape

torch.Size([50, 74, 4, 8])